<a href="https://colab.research.google.com/github/dapsavoie/tz_water_well_prediction/blob/master/74%25_Submitted_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries 

In [0]:
import pandas as pd 
import numpy as np

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Train dummies 
train_dummies = pd.read_csv(r'/content/drive/My Drive/tz_water_well_data/train_dummmies.csv')

#Test export to csv 
test_dummies = pd.read_csv(r'/content/drive/My Drive/tz_water_well_data/test_dummies.csv')

#df_lables
df_labels = pd.read_csv('/content/drive/My Drive/tz_water_well_data/training_set_values.csv')

# Xgboost Pipeline

In [0]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

from sklearn.pipeline import Pipeline


In [0]:
#Split predictor and target variables 

y = df_labels['status_group']
X = train_dummies 

In [0]:
#split into predictor and target variables 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [68]:
scaler=StandardScaler()

#transform training and test 
scaled_data_train = scaler.fit_transform(X_train)
scaled_data_test = scaler.fit_transform(X_test)

#Convert to data dataframe 
scaled_df_train = pd.DataFrame(scaled_data_train, columns=X_train.columns)
scaled_df_train.head()

,Unnamed: 0,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,funder_Danida,funder_Government of Tanzania,funder_Hesawa,funder_Rwssp,funder_World_Bank,funder_other,installer_Commu,installer_DANIDA,installer_DWE,installer_Danida,installer_RWE,installer_other,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Arusha,region_Dar es Salaam,region_Dodoma,region_Iringa,region_Kagera,region_Kigoma,region_Kilimanjaro,region_Lindi,...,quantity_seasonal,quantity_unknown,quantity_group_dry,quantity_group_enough,quantity_group_insufficient,quantity_group_seasonal,quantity_group_unknown,source_dam,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,source_type_borehole,source_type_dam,source_type_other,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,-1.522547,-1.709469,-0.084999,2.053863,0.204135,0.501977,-0.03807,0.322017,-0.480201,-0.041306,0.436109,-0.235052,-0.424305,-0.198075,-0.154438,-0.152209,0.636953,-0.134955,-0.133738,-0.644435,-0.175634,-0.144505,0.781672,2.575842,-0.303748,-0.208844,-0.349359,-0.456836,-0.420604,-0.393624,-0.286299,-0.335356,-0.245055,-0.11813,-0.195827,-0.313563,-0.243604,-0.223199,-0.280493,-0.164363,...,-0.269828,-0.115819,-0.343836,-1.124633,1.710548,-0.269828,-0.115819,-0.10364,-0.122362,-0.113654,-0.478359,-0.061318,-0.200074,-0.440386,-0.631245,1.584418,-0.034349,-0.501512,-0.10364,-0.070346,-0.200074,-0.460868,-0.631245,1.584418,0.545347,-0.537743,-0.070346,-0.043802,1.041913,-0.336365,-0.009175,-0.648656,-0.116749,-0.346660,-0.043802,0.849002,-0.009175,-0.648656,-0.116749,-0.346660
1,1.237767,-1.706858,-0.100621,-0.965049,0.216991,-0.006643,-0.03807,-0.813147,0.037334,-0.379739,-0.623675,-0.235052,-0.424305,-0.198075,-0.154438,-0.152209,0.636953,-0.134955,-0.133738,-0.644435,-0.175634,-0.144505,0.781672,2.575842,-0.303748,-0.208844,-0.349359,-0.456836,-0.420604,-0.393624,-0.286299,-0.335356,-0.245055,-0.11813,5.106557,-0.313563,-0.243604,-0.223199,-0.280493,-0.164363,...,-0.269828,-0.115819,-0.343836,0.889179,-0.584608,-0.269828,-0.115819,-0.10364,-0.122362,-0.113654,-0.478359,-0.061318,-0.200074,-0.440386,1.584172,-0.631147,-0.034349,-0.501512,-0.10364,-0.070346,-0.200074,-0.460868,1.584172,-0.631147,0.545347,-0.537743,-0.070346,-0.043802,-0.959773,-0.336365,-0.009175,1.541648,-0.116749,-0.346660,-0.043802,-1.177854,-0.009175,1.541648,-0.116749,-0.346660
2,-0.541022,-1.071015,-0.100621,-0.965049,-0.243403,-1.147020,-0.03807,-0.188807,0.037334,-0.379739,-0.623675,-0.235052,-0.424305,-0.198075,-0.154438,-0.152209,0.636953,-0.134955,-0.133738,-0.644435,-0.175634,-0.144505,0.781672,-0.388223,-0.303748,4.788266,-0.349359,-0.456836,-0.420604,-0.393624,-0.286299,-0.335356,-0.245055,-0.11813,-0.195827,-0.313563,-0.243604,-0.223199,-0.280493,-0.164363,...,-0.269828,-0.115819,-0.343836,0.889179,-0.584608,-0.269828,-0.115819,-0.10364,-0.122362,-0.113654,-0.478359,-0.061318,-0.200074,-0.440386,1.584172,-0.631147,-0.034349,-0.501512,-0.10364,-0.070346,-0.200074,-0.460868,1.584172,-0.631147,0.545347,-0.537743,-0.070346,-0.043802,-0.959773,-0.336365,-0.009175,-0.648656,-0.116749,2.884667,-0.043802,-1.177854,-0.009175,-0.648656,-0.116749,2.884667
3,1.350836,0.480077,-0.100621,-0.965049,-0.004716,-1.061741,-0.03807,-0.188

In [69]:
clf = xgb.XGBClassifier()

#fit
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
#build a pipeline 

scaled_pipeline_1 = Pipeline([('ss', StandardScaler()),
                               ('xgb', xgb.XGBClassifier())])

In [71]:
#fit to train 

# Fit the training data to pipeline
scaled_pipeline_1.fit(X_train, y_train)

# Print the accuracy on test set
scaled_pipeline_1.score(X_test, y_test)

0.7478114478114478

# XGBOOST

In [0]:
labels = df_labels['status_group']

x = train_dummies

In [0]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

In [0]:
scaler = StandardScaler()

In [0]:
scaled_df = scaler.fit_transform(x)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_df, labels)

In [77]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 75.09%
Validation accuracy: 75.25%


In [87]:
test_dummies = scaler.fit_transform(test_dummies)
test_preds = clf.predict(test_dummies, validate_features=False)
val_accuracy = accuracy_score(y_test, test_preds)
val_accuracy

0.5008754208754209

In [0]:
predictions = test_preds
test_preds = pd.DataFrame(test_preds)
submission = pd.DataFrame(y_test.join(test_preds))
submission.rename(columns={0: 'status_group'}, inplace=True)

In [0]:
submission.to_csv('/content/drive/My Drive/tz_water_well_data/buddha.csv' ,index=0)

# Xgboost

In [0]:
# Grid Search CV 

from sklearn.model_selection import GridSearchCV

In [0]:
param_grid = {
    "learning_rate": [0.1], #choose weights that minimize cost function
    'max_depth': [6], 
    'min_child_weight': [10],
    'subsample': [ 0.7],
    'n_estimators': [5, 30, 100, 250],
}

In [0]:
grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=3, n_jobs=1)
grid_clf.fit(scaled_df, labels)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
val_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

# 
